# Turkish NLP Pipeline with BERT 

1. Sentiment Analysis
2. NER Model
3. Question Answering
4. Text Summarization
5. Text Categorization

These models are fined tuned based on Turkish-Bert model 

https://github.com/stefan-it/turkish-bert

## Requirements
* python3
* pip install transformers

## 1-Sentiment Analysis
* This model can get up to %95 success rate on our dataset 
* To see the training detail and the model performce, check the link \
 https://huggingface.co/savasy/bert-base-turkish-sentiment-cased

### Usage

In [66]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
# load model, it takes time since it load over 500 MB model file
model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
sa= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

### Example usage

In [2]:
p= sa("bu telefon modelleri çok kaliteli , her parçası çok özel bence")
print(p)
#[{'label': 'LABEL_1', 'score': 0.9871089}]
print (p[0]['label']=='LABEL_1')
#True


[{'label': 'LABEL_1', 'score': 0.9967432}]
True


In [3]:
p= sa("Film çok kötü ve çok sahteydi")
print(p)
#[{'label': 'LABEL_0', 'score': 0.9975505}]
print (p[0]['label']=='LABEL_1')
#False

[{'label': 'LABEL_0', 'score': 0.99860054}]
False


### Test your entire dataset

> Suppose your file has lots of lines of comment and label (1 or 0) at the end (tab seperated)

```
 # yourfile.tsv
 comment1 ... \t label
 comment2 ... \t label

 comment-n ... \t  label
...
```



```
# your test code

from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

f="/path/to/your/file/yourfile.tsv"
model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
sa= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

i,crr=0,0
for line in open(f):
 lines=line.strip().split("\t")
 if len(lines)==2:
  i=i+1
  if i%100==0:
   print(i)
  pred= sa(lines[0])
  pred=pred[0]["label"].split("_")[1]
  if pred== lines[1]:
   crr=crr+1

print(crr, i, crr/i)
```

# 2- Name Entity Recognizer (NER)
This model can get %95 accuracy, currently it works with PER, LOC, and ORG

check model repo for other detail

* https://huggingface.co/savasy/bert-base-turkish-ner-cased


## Example usage

In [51]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")

ner=pipeline('ner', model=model, tokenizer=tokenizer)
ner("Mustafa Kemal Atatürk 19 Mayıs 1919'da Samsun'a ayak bastı.")

[{'word': 'Mustafa', 'score': 0.9938516616821289, 'entity': 'B-PER'},
 {'word': 'Kemal', 'score': 0.9881671071052551, 'entity': 'I-PER'},
 {'word': 'Atatürk', 'score': 0.9957979321479797, 'entity': 'I-PER'},
 {'word': 'Samsun', 'score': 0.9059973359107971, 'entity': 'B-LOC'}]

In [5]:
ner("Sait Faik ömrünün sonuna kadar yazları Burgaz adadaki köşklerinde kalmıştır")

[{'word': 'Sait', 'score': 0.9995592832565308, 'entity': 'B-PER'},
 {'word': 'Faik', 'score': 0.999716579914093, 'entity': 'I-PER'},
 {'word': 'Bur', 'score': 0.7886330485343933, 'entity': 'B-LOC'},
 {'word': '##gaz', 'score': 0.8438959121704102, 'entity': 'I-LOC'}]

# 3- Question Answering  (SQuAD) for Turkish

This model istranied with TQuAD dataset (which is SQuAD version of Turkish)

https://github.com/TQuad/turkish-nlp-qa-dataset


In [52]:
# load model

In [53]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-squad")
model = AutoModelForQuestionAnswering.from_pretrained("savasy/bert-base-turkish-squad")
nlp=pipeline("question-answering", model=model, tokenizer=tokenizer)

In [57]:
sait="ABASIYANIK, Sait Faik. Hikayeci (Adapazarı 23 Kasım 1906-İstanbul 11 Mayıs 1954). \
İlk öğrenimine Adapazarı’nda Rehber-i Terakki Mektebi’nde başladı. İki yıl kadar Adapazarı İdadisi’nde okudu.\
İstanbul Erkek Lisesi’nde devam ettiği orta öğrenimini Bursa Lisesi’nde tamamladı (1928). İstanbul Edebiyat \
Fakültesi’ne iki yıl devam ettikten sonra babasının isteği üzerine iktisat öğrenimi için İsviçre’ye gitti. \
Kısa süre sonra iktisat öğrenimini bırakarak Lozan’dan Grenoble’a geçti. Üç yıl başıboş bir edebiyat öğrenimi \
gördükten sonra babası tarafından geri çağrıldı (1933). Bir müddet Halıcıoğlu Ermeni Yetim Mektebi'nde Türkçe \
gurup dersleri öğretmenliği yaptı. Ticarete atıldıysa da tutunamadı. Bir ay Haber gazetesinde adliye muhabirliği\
yaptı (1942). Babasının ölümü üzerine aileden kalan emlakin geliri ile avare bir hayata başladı. Evlenemedi.\
Yazları Burgaz adasındaki köşklerinde, kışları Şişli’deki apartmanlarında annesi ile beraber geçen bu fazla \
içkili bohem hayatı ömrünün sonuna kadar sürdü."

In [58]:
print(nlp(question="Ne zaman avare bir hayata başladı?", context=sait))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4025.24it/s]


{'score': 0.7435662892824979, 'start': 752, 'end': 775, 'answer': 'Babasının ölümü üzerine'}


In [59]:
print(nlp(question="Sait Faik hangi Lisede orta öğrenimini tamamladı?", context=sait))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5165.40it/s]


{'score': 0.32522039650666557, 'start': 246, 'end': 262, 'answer': 'Bursa Lisesi’nde'}


In [60]:
print(nlp(question="Sait Faik bir film yönetmeni mi?", context=sait))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3768.47it/s]


{'score': 0.0804725005957514, 'start': 498, 'end': 515, 'answer': 'edebiyat öğrenimi'}


In [61]:
# Ask your self ! type your question
print(nlp(question="...?", context=sait))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2562.19it/s]


{'score': 0.013415400836009894, 'start': 0, 'end': 11, 'answer': 'ABASIYANIK,'}


In [62]:
ataturk="Atatürk  modern, ilerici ve laik bir ulus devleti oluşturmak için politik, ekonomik ve kültürel alanlarda sekülarist ve milliyetçi \
 karakterdeki reformlarını başlattı. Yabancılara tanınan ekonomik imtiyazlar kaldırıldı ve onlara ait üretim araçları ve demiryolları millîleştirildi. \
 Tevhîd-i Tedrîsât Kanunu ile eğitim Türk hükûmetinin denetimine girdi. Seküler ve bilimsel eğitim esas alındı. Binlerce yeni okul inşa edildi. \
 İlköğretim ücretsiz ve zorunlu hale getirildi. Yabancı okullar devlet denetimine alındı. Köylülerin sırtına yüklenen ağır vergiler azaltıldı. \
 Erkeklerin serpuş ve kıyafetlerinde değişiklikler yapıldı. Takvim, saat ve ölçülerde değişikliklere gidildi. \
 Mecelle kaldırılarak yerine seküler Türk Kanunu Medenisi yürürlüğe konuldu. Kadınların sivil ve politik hakları pek çok Batı ülkesinden önce tanındı. \
 Çok eşlilik yasaklandı. Kadınların şahitliği ve miras hakkı erkeklerinkiyle eşit hale getirildi. Benzer şekilde, dünyanın çoğu ülkesinden önce olarak \
 Türkiye'de kadınların ilkin yerel seçimlerde (1930), sonra genel seçimlerde (1934) seçme ve seçilme hakkı tanındı. Ceza ve borçlar hukukunda \
 seküler yasalar yürürlüğe konuldu. Sanayi Teşvik Kanunu kabul edildi. Toprak Reformu için çabalandı.[3] Arap harfleri temelli Osmanlı alfabesinin yerine \
  Latin harfleri temelli yeni Türk alfabesi kabul edildi. Halkı okuryazar kılmak için eğitim seferberliği başlatıldı. Üniversite Reformu gerçekleştirildi. \
  Birinci Beş Yıllık Sanayi Planı yürürlüğe konuldu. Sınıf ve statü farkı gözeten lâkap ve unvanlar kaldırıldı ve soyadları yürürlüğe konuldu. \
  Homojen ve birleşmiş bir ulus yaratılması için Türkleştirme politikası yürütüldü.[4][5][6] Türk olmayan azınlıklar kamuoyunda Türkçe konuşmaya zorlandı,[7] \
  Türkçe olmayan toponomiler ve azınlıkların soyadları Türkçeye çevrildi.[8][9]"

In [63]:
print(nlp(question="Kimlere ekonomik imtiyaz kaldırıldı?", context=ataturk))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4821.04it/s]


{'score': 0.17692459754713763, 'start': 168, 'end': 179, 'answer': 'Yabancılara'}


In [64]:
print(nlp(question="tüm bu devrimleri kim yaptı?", context=ataturk))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4116.10it/s]


{'score': 4.634995517165336e-07, 'start': 0, 'end': 8, 'answer': 'Atatürk'}


In [65]:
# Ask your self ! type your question
print(nlp(question="...?", context=ataturk))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3606.45it/s]


{'score': 0.0014260291964551947, 'start': 1797, 'end': 1811, 'answer': 'çevrildi.[8][9]'}


# 4 - Text Summarization

...will be soon

# 5- Text Categorization

 .. will be soon
